In [5]:
%load_ext autoreload
%autoreload 2
import jieba
import jieba.posseg as pseg
import nltk
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import torch
import re
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import utils.LS_pipeline as LS_pipeline
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-news')


2025-02-21 09:40:41,891 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-21 09:40:45,971 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-21 09:40:46,537 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:40:46,537 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-21 09:40:46,552 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:40:49,110 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
2025-02-21 09:40:49,628 - modelscope - INFO - All model checkpoint weights were used when initializing ModelForTokenClassificationWithCRF.

2025-02-21 09:40:49,630 - modelscope - INFO - All the weights of ModelForTokenClassificationWithCRF were initialized from the model checkpoint If your task is similar to the t

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-02-21 09:40:52,626 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-21 09:40:58,320 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-21 09:40:58,777 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:40:58,779 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-21 09:40:58,787 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:41:01,641 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [2]:
with open("../data/BLCU/top_similar.pickle", 'rb') as handle:
    top_similar = pickle.load(handle)
with open("../data/BLCU/similarity_dict.pickle", 'rb') as handle:
    similarity_dict = pickle.load(handle)
with open("../data/HSK/HSK_levels.pickle", 'rb') as handle:
    hsk_dict = pickle.load(handle)

blcu = pd.read_csv('../data/BLCU/literature_wordfreq.release_UTF-8.txt', header = None, sep="\t",)
blcu.rename(columns={0:"character", 1:"frequency"}, inplace=True)
blcu.set_index("character", inplace=True)
blcu["frequency"] = blcu["frequency"].rank(pct=True)


In [6]:
sentence = "他的见解十分独到，对问题的分析非常精辟。"
sentence = "十个科技业被选定为中国到2025年应主导的重点领域。"
LS_pipeline.LS_pipeline(sentence, verbose=True)

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


ner time is: 1.4026978999900166
complex_word time is: 0.3842667999997502
	 Entering choose func...
cand embed time: 0.32935929999803193
cos sim time: 0.0006557999877259135
cand embed time: 0.4394475000008242
cos sim time: 0.0005821000086143613
cand embed time: 0.3471447000047192
cos sim time: 0.0008817999914754182
cand embed time: 0.3630323999968823
cos sim time: 0.0010704000014811754
cand embed time: 0.3168957000016235
cos sim time: 0.0006098000012570992
cand embed time: 0.3697461999981897
cos sim time: 0.0007464000082109123
cand embed time: 0.36935509998875204
cos sim time: 0.0006308000010903925
cand embed time: 0.3474056999984896
cos sim time: 0.0007915000023785979
choose time is: 3.5504481000098167
	 Entering choose func...
cand embed time: 0.33010760000615846
cos sim time: 0.0006073999975342304
cand embed time: 0.311628199997358
cos sim time: 0.000564800007850863
cand embed time: 0.3217648999998346
cos sim time: 0.000679899996612221
cand embed time: 0.3123500000074273
cos sim time

'十个科技业被选为中国到2025年应主导的重点范畴。'

In [7]:
"""Pipeline for lexical simplification (LS) task using HSK dictionary lookup and best candidate replacement."""

###### import packages, NER pipeline
import re
import jieba
import torch
import string
import pandas as pd
import numpy as np
import jieba.posseg as pseg
import pickle
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-news')

from time import perf_counter

###### Load vocab data
with open("../data/BLCU/similarity_dict_v2.pickle", 'rb') as handle:
    similarity_dict = pickle.load(handle)
with open("../data/HSK/HSK_levels.pickle", 'rb') as handle:
    hsk_dict = pickle.load(handle)
# punc = ",!?！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.-" # possible punctuation
blcu = pd.read_csv('../data/BLCU/literature_wordfreq.release_UTF-8.txt', header = None, sep="\t",)
blcu.rename(columns={0:"character", 1:"frequency"}, inplace=True)
blcu.set_index("character", inplace=True)
blcu["frequency"] = blcu["frequency"].rank(pct=True)

2025-02-21 09:48:21,984 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-02-21 09:48:25,068 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-21 09:48:25,507 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:48:25,509 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-21 09:48:25,519 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:48:28,507 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

2025-02-21 09:48:37,711 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-02-21 09:48:38,126 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:48:38,126 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-02-21 09:48:38,144 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-02-21 09:48:40,795 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [8]:
def find_complex_words(tokens: list, HSK_thresh:int = 5, freq_thresh:float = 0.98):
    """Finds complex words based on one of two criteria: HSK level, and frequency in BLCU data. Assumes
    less frequent words are more difficult or less likely to be known."""
    complex_HSK = [token for token in tokens if (token in hsk_dict and hsk_dict[token]>HSK_thresh)] # find all tokens above HSK level
    simple_HSK = [token for token in tokens if (token in hsk_dict and hsk_dict[token]<=HSK_thresh)] # also find simple tokens
    complex_freq = [token for token in tokens if (token in blcu.index and blcu.loc[token].values[0]<freq_thresh)] # find all tokens below freq level
    complex_words = list(set(complex_HSK).union(set(complex_freq).difference(set(simple_HSK)))) # combine by union
    return complex_words

In [48]:
tokens = jieba.lcut(sentence)
tokens = [token for token in tokens if not re.match(r'^\W+$', token)] # remove punctuation

### NER on each token:
ner_output = ner_pipeline(sentence)['output'] # run NER pipeline, generate tokens
tokens_ner = list(set([d['span'] for d in ner_output if len(d['span'])>1])) # only collect NER longer than 1 character
tokens_no_ner = list(set(tokens) - set(tokens_ner)) # get all tokens that are NOT named entities

### get complex words:
complex_words = find_complex_words(tokens_no_ner)

### get candidates and build new sentences:
simple_sentence = sentence
candidates = [[word]+similarity_dict[word] for word in complex_words]

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [68]:
sentences = []
for word in candidates:
    sentences += [sentence.replace(word[0], candidate) for candidate in word] # new sentence with word replaced

In [69]:
sentences

['十个科技业被选定为中国到2025年应主导的重点领域。',
 '十个科技业被选定为中国到2025年应主导的重点领域。',
 '十个科技业被选中为中国到2025年应主导的重点领域。',
 '十个科技业被选出为中国到2025年应主导的重点领域。',
 '十个科技业被选上为中国到2025年应主导的重点领域。',
 '十个科技业被决定为中国到2025年应主导的重点领域。',
 '十个科技业被指定为中国到2025年应主导的重点领域。',
 '十个科技业被选取为中国到2025年应主导的重点领域。',
 '十个科技业被选为中国到2025年应主导的重点领域。',
 '十个科技业被选择为中国到2025年应主导的重点领域。',
 '十个科技业被选用为中国到2025年应主导的重点领域。',
 '十个科技业被定夺为中国到2025年应主导的重点领域。',
 '十个科技业被自选为中国到2025年应主导的重点领域。',
 '十个科技业被挑选为中国到2025年应主导的重点领域。',
 '十个科技业被选派为中国到2025年应主导的重点领域。',
 '十个科技业被选定为中国到2025年应主导的重点领域。',
 '十个科技业被选定为中国到2025年应主导的重点领域。',
 '十个科技业被选定为中国到2025年应主导的重点区域。',
 '十个科技业被选定为中国到2025年应主导的重点地域。',
 '十个科技业被选定为中国到2025年应主导的重点范畴。',
 '十个科技业被选定为中国到2025年应主导的重点行业。',
 '十个科技业被选定为中国到2025年应主导的重点类别。',
 '十个科技业被选定为中国到2025年应主导的重点范围。',
 '十个科技业被选定为中国到2025年应主导的重点学科。',
 '十个科技业被选定为中国到2025年应主导的重点流域。',
 '十个科技业被选定为中国到2025年应主导的重点水域。',
 '十个科技业被选定为中国到2025年应主导的重点场合。',
 '十个科技业被选定为中国到2025年应主导的重点部门。',
 '十个科技业被选定为中国到2025年应主导的重点地方。',
 '十个科技业被选定为中国到2025年应主导的重点单位。']

In [70]:
embed_new = model.encode(sentences, batch_size=32, convert_to_tensor=True) # create embeddings

In [28]:
embed_new.shape

(14, 1024)

In [ ]:
similarity = []
tstart = perf_counter()
embed_new = model.encode(new_sentences) # create embeddings
print("cand embed time:", perf_counter()-tstart)
similarity.append(np.float32(model.similarity(embed_orig, embed_new)[0][0])) # calculate similarity
best_idx = int(np.argmax(similarity))
simple_sentence = sentence.replace(word, candidates[best_idx])